
## Simple Draft for the implementation of LDPC Encoder based on Wimax Standard

This draft is for the developer who wants to quickly understand the structure of LDPC Encoder based on Wimax protocol and also a brief instruction for the implementation of LDPC hardware encoder in verilog. If not specificly assigned, the LDPC Encoder here only means under the standard Wimax.The requirement of high throughput and low latency should be fullfilled. 

The consideration here for hardward encoder is based on pure verilog which means that the encoder doesn't use any library for FPGA or some other platforms. The integration to specific hardware platform is possible and should be considered by the developer. The draft is based on personal work as a research helper.

### Theory of LDPC Encoder

LDPC code is the abbreviation of low-density parity-check code which is a linear error correcting code, a method of transmitting a message over a noisy transmission channel. The parity check matrix of LDPC is sparse and means "low density". The encoding is based on this low density parity check matrix( using H to represent LDPC parity check matrix). 

LDPC code is systematic code, which means that the information contains in the encoded codeword. The codeword is represented by 
$v = {u, p1, p2}$, in which $u$ is the original data, $p1$ and $p2$ represent the parity check bits. The codeword should satisfy the equation $$v \cdot H^T = 0$$
The encoding procedure is actually to calculate $p1$ and $p2$, and then conbine $p1$ and $p2$ with $u$ to generate the codeword $v$.

### LDPC Base Matrix
There are six base matrixs for six different code rate, which are 1/2, 2/3(A), 2/3(B), 3/4(A), 3/4(B) and 5/6. For simple explain, the code rate 1/2(Class A) will be used here.The base matrix of 1/2 code rate is shown as below:

In [1]:
'''

Python Program for LDPC Encoder 
based on Wimax 802.16e Standard


'''
import numpy as np

baseMatrix = np.array([
[-1,94,73,-1,-1,-1,-1,-1,55,83,-1,-1, 7, 0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
[-1,27,-1,-1,-1,22,79, 9,-1,-1,-1,12,-1, 0, 0,-1,-1,-1,-1,-1,-1,-1,-1,-1],
[-1,-1,-1,24,22,81,-1,33,-1,-1,-1, 0,-1,-1, 0, 0,-1,-1,-1,-1,-1,-1,-1,-1],
[61,-1,47,-1,-1,-1,-1,-1,65,25,-1,-1,-1,-1,-1, 0, 0,-1,-1,-1,-1,-1,-1,-1],
[-1,-1,39,-1,-1,-1,84,-1,-1,41,72,-1,-1,-1,-1,-1, 0, 0,-1,-1,-1,-1,-1,-1],
[-1,-1,-1,-1,46,40,-1,82,-1,-1,-1,79, 0,-1,-1,-1,-1, 0, 0,-1,-1,-1,-1,-1],
[-1,-1,95,53,-1,-1,-1,-1,-1,14,18,-1,-1,-1,-1,-1,-1,-1, 0, 0,-1,-1,-1,-1],
[-1,11,73,-1,-1,-1, 2,-1,-1,47,-1,-1,-1,-1,-1,-1,-1,-1,-1, 0, 0,-1,-1,-1],
[12,-1,-1,-1,83,24,-1,43,-1,-1,-1,51,-1,-1,-1,-1,-1,-1,-1,-1, 0, 0,-1,-1],
[-1,-1,-1,-1,-1,94,-1,59,-1,-1,70,72,-1,-1,-1,-1,-1,-1,-1,-1,-1, 0, 0,-1],
[-1,-1, 7,65,-1,-1,-1,-1,39,49,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1, 0, 0],
[43,-1,-1,-1,-1,66,-1,41,-1,-1,-1,26, 7,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1, 0]])

rows = baseMatrix.shape[0]
cols = baseMatrix.shape[1]
# expansion factor
z = 24


Each element in the matrix represent a identity matrix or a shifted identity matrix with size $z * z$, $z$ is expansion factor. The value of z and corresponding code length and code rate is showed in below.$n$ is code length, $z$ is factor and $R$ represents code rate.
![](ldpc_table.png)  
![](ldpc_table2.png)  
For the element value, $e$ represent a element in the matrix.  

if(e < 0): e is all-zero matrix.  
else if(e == 0): e is identity matrix  
else: e is a shifted identity matrix with shift param $d$   

The calculation of shifted identity matrix is easy.As in below, an identity matrix is like:

In [2]:
identity_matrix = np.eye(5)
d = 3
tmp = identity_matrix[0:d]
shifted_identity_matrix = np.vstack((identity_matrix[d:5], tmp))

In [3]:
identity_matrix

array([[ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]])

If now the shift param $d$ is 3, then the new matrix will look like:

In [4]:
shifted_identity_matrix

array([[ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.]])

The values in the base matrix actually means some kind of shifts like this, but the value $d$ needs to be pre-calculated from the base matrix according to different code rate. According to the Wimax Standard in 2012, the shift parameter $d$ is should be calculated for different code rate.  
For rate 2/3 A code, 
$$
d = \left\{\begin{matrix}
d  ,      &  &  d\leq 0  &  & \\ 
 mod(d, z)  &  & d> 0  &  &  &  & 
\end{matrix}\right.
$$

For the other code rate,
$$
d = \left\{\begin{matrix}
 d  ,      &  &  d\leq 0  &  & \\ 
 \left \lfloor \frac{d\cdot z}{z_{0}} \right \rfloor  &  & d> 0  &  &  &  & 
\end{matrix}\right.
$$
Here $z_{0}$ equals 96, $z$ corresponds to different requirements for code length.

### LDPC Encoding Algorithm

The encoding algorithm here used is called RU Algorithm, which is described in this article "Efficient encoding of low-density parity-check codes" by T.J. Richardson and R.L. Urbanke. The Algorithm is easy enough so it's alse called "direct encoding". The algorithm is based on Gaussian Elimination. The detailed explanation of this will not be discussed here, which could be found totally in the paper. 
The algorithm works in several steps and then it calculates $p_{1}$ and $p_{2}$. For the calculation, the base matrix should be divided into six submatrixs, namely A, B, C, D, T, E.
![](submat_ldpc.png)

"0" in the top-right corner represents all-zero matrix. This form of matrix is called lower trianglar form, which is efficient for Gaussian Elimination. The submatrixs are showed in below. Here $q$ equals 24, $p$ equals 12, $g$ equals 1.


In [5]:
A_Base = np.zeros((11,12),dtype=int)
B_Base = np.zeros((11,1),dtype=int)
T_Base = np.zeros((11,11),dtype=int)
for i in range(rows-1):
    A_Base[i] = baseMatrix[i][0:rows]
    B_Base[i] = baseMatrix[i][rows:rows+1]
    T_Base[i] = baseMatrix[i][rows+1:cols]
    
C_Base = baseMatrix[rows-1][0:rows].reshape(1,rows)
D_Base = baseMatrix[rows-1][rows:rows+1].reshape(1,1)
E_Base = baseMatrix[rows-1][rows+1:cols].reshape(1,rows-1)

In [6]:
A_Base

array([[-1, 94, 73, -1, -1, -1, -1, -1, 55, 83, -1, -1],
       [-1, 27, -1, -1, -1, 22, 79,  9, -1, -1, -1, 12],
       [-1, -1, -1, 24, 22, 81, -1, 33, -1, -1, -1,  0],
       [61, -1, 47, -1, -1, -1, -1, -1, 65, 25, -1, -1],
       [-1, -1, 39, -1, -1, -1, 84, -1, -1, 41, 72, -1],
       [-1, -1, -1, -1, 46, 40, -1, 82, -1, -1, -1, 79],
       [-1, -1, 95, 53, -1, -1, -1, -1, -1, 14, 18, -1],
       [-1, 11, 73, -1, -1, -1,  2, -1, -1, 47, -1, -1],
       [12, -1, -1, -1, 83, 24, -1, 43, -1, -1, -1, 51],
       [-1, -1, -1, -1, -1, 94, -1, 59, -1, -1, 70, 72],
       [-1, -1,  7, 65, -1, -1, -1, -1, 39, 49, -1, -1]])

In [7]:
B_Base

array([[ 7],
       [-1],
       [-1],
       [-1],
       [-1],
       [ 0],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1]])

In [8]:
C_Base

array([[43, -1, -1, -1, -1, 66, -1, 41, -1, -1, -1, 26]])

In [9]:
D_Base

array([[7]])

In [10]:
E_Base

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0]])

In [11]:
T_Base

array([[ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1,  0,  0, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1,  0,  0, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1,  0,  0, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1,  0,  0, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1,  0,  0, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1,  0,  0, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1,  0,  0, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1,  0,  0, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  0]])

In [12]:
# Helper function for calculating whole matrix, call gen_whole_mat(matrix to be converted, expansion factor)
def gen_sub_mat(val, z):
    if(val == -1):
        return np.zeros((z,z), dtype=int)
    else:
        sub_mat = np.eye(z,dtype=int)
        rest = val % z
        
        tmp = sub_mat[0:rest]
        return np.vstack((sub_mat[rest:z], tmp))

def gen_whole_mat(Mat_Base, z):
    rows, cols = Mat_Base.shape
    
    for i in range(0, rows):
        for j in range(0,cols):
            if j == 0:
                tmp = gen_sub_mat(Mat_Base[i][j], z)
            else:
                tmp = np.hstack((tmp, gen_sub_mat(Mat_Base[i][j], z)))
        if i == 0:
            temp = tmp
        else:
            temp = np.vstack((temp, tmp))
    return temp


The submatrixs could be converted easily into whole matrix, which means the matrix of 0 and 1. The dimension is large. When $z$ equals 24, the code lenth is 576 and the dimension of submatrix A is 264 * 288. In order to do the algorithm, all base matrixs A, B, C, D, E, T should be firstly converted to whole matrixs. In below, A, B, C, D, E, T represent the whole matrix. The algorithm works in seven steps:  
1. $f_{1} = A \cdot S^{T}$  
2. $f_{2} = C \cdot S^{T}$  
3. $f_{3} = E \cdot T^{-1} \cdot f_{1}$  
4. $p_{1} = f_{3} + f_{2}$  
5. $f_{4} = B \cdot p_{1}^{T}$  
6. $f_{5} = f_{1} + f_{4}$  
7. $p_{2} = T^{-1} \cdot f_{5}$  

After $p_{1}$ and $p_{2}$ are calculated, the codeword is {$u, p_{1}^{T}, p_{2}^{T}$}

The encoder implementation in Python is showed in below:

In [13]:
# Whole Matrix of A, B, C, D, E, T
A_Whole = gen_whole_mat(A_Base, 24)
B_Whole = gen_whole_mat(B_Base, 24)
C_Whole = gen_whole_mat(C_Base, 24)
D_Whole = gen_whole_mat(D_Base, 24)
E_Whole = gen_whole_mat(E_Base, 24)
T_Whole = gen_whole_mat(T_Base, 24)

# T inverse and E*(T inverse)
TT_Whole = np.mat(T_Whole, dtype=int).I.astype(int)
ETT_Whole = np.dot(E_Whole, TT_Whole)

# Encoding
len = 12 * z

data_placeholder = np.zeros(len,dtype=int).reshape(1,len)

for i in range(0,len):
    data_placeholder[0][i] = np.random.randint(0,2)

data = np.mat(data_placeholder)
# Algorithm steps
f1 = np.dot(A_Whole, data.T) % 2
f2 = np.dot(C_Whole, data.T) % 2
f3 = np.dot(ETT_Whole, f1) % 2
p1 = f3 + f2

f4 = np.dot(B_Whole, p1) % 2
f5 = f1 + f4
p2 = np.dot(TT_Whole, f5) % 2
# Codeword
codeword = np.hstack((data, p1.T, p2.T))

### Hardward Implementation of LDPC Encoder
The hardware implementation structure is designed as below.
![](hardldpc.png)

After combining $u$, $p_{1}$, $p_{2}$, the codeword is generated as {$u, p_{1}, p_{2}$}

### Consideration of Parallelization

In order to fullfill the requirements of high throughput and low latency, the structure need to be parallelized. The most time consuming part is multiply, so the consideration here is to parallelize the multiply of matrix. Consider about the sparsity of matrix, the multiply could be done very quickly.

As an easy example, if we want to calculate $M \cdot N$, in which

In [14]:
M = np.array([[1, 0, 0, 1, 0],
              [1, 0, 0, 0, 1],
              [0, 0, 0, 1, 1],
              [0, 1, 0, 0, 0],
              [0, 0, 1, 1, 0]])

In [15]:
M

array([[1, 0, 0, 1, 0],
       [1, 0, 0, 0, 1],
       [0, 0, 0, 1, 1],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 1, 0]])

In [16]:
N = np.array([[1],[0],[0],[1],[0]])

In [17]:
N

array([[1],
       [0],
       [0],
       [1],
       [0]])

The result is:

In [18]:
result = M.dot(N) % 2

In [19]:
result

array([[0],
       [1],
       [1],
       [0],
       [1]], dtype=int32)


The first row of the result $\ result_{0} = N[0]\ xor\  N[3] == 0$, which is the xor of non-zero index in the first row of matrix $M$  
Similarly, the second row $\ result_{1} = N[0]\ xor\ N[4] == 1$  
Now if we want to calculate $A \cdot s^{T}$ in the fisrt step of the algorithm, the most efficient way is to find the non-zero position of the first line in matrix $A$, and then xor them. In the base matrix $A$, the non-zero elements are 94, 73, 55, 83. When the expansion factor $z$ equals to 24, the non-zero position could be calculated which equals to 47, 66, 205, 236 respectively. So the first line of multiplication of $A \cdot s^{T}$ equals to $$ result_{0} = s[47]\ xor\ s[66]\ xor\ s[205]\ xor\ s[236]$$
The calculation of every line in matrix $A$ is independent, which means that the calculation could be done parallelly. All multiplications could be done in the same way. For this method, the position of non-zero position in each submatrix needed to be pre-calculated. This is simple by using matlab or python.

In order to improve the throughput, some optimization should be considered and the structure should also be pipelined as followed.
![](pipe.png)

The blue path is the critical path of the circuit. In order to improve the throughput, two registers need to be placed. If the algorithm is fully parallelized, the register is mainly for storing the temp data and wait until the calculation from the critical path is finished. After $p1$ and $p2$ are calculated, the codeword is {$u, p1, p2$}.